In [15]:
#!pip install sentinelhub

# Commercial data

For use of commercial data you have to first provide your sentinel-hub account client id and client secret.

In [9]:
from sentinelhub import SHConfig
from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from SentinelHubCommercialData import SentinelHubCommercialData

config = SHConfig()
config.sh_client_id = '792571cd-7ee1-456d-bdc7-7334e6b610f0'
config.sh_client_secret = ')Kw:<ZC1975EsLNx6R-c65[[-+X3C]/Yp8-<#!p5'
config.sh_base_url = "https://services.sentinel-hub.com"

shcd = SentinelHubCommercialData(config)

## Getting quotas

In [10]:
# Optional 1: To get single quota add id as parameter
#
#     shcd.quotas("asd124-12ddas...")
#
# Optional 2: Get raw response
#
#     shcd.quotas(raw=True)
#

q = shcd.quotas()
print(q)



AIRBUS_SPOT
Quota: 1000.0
Used: 14.5
----------
AIRBUS_PLEIADES
Quota: 100.0
Used: 7.0
----------



# Simple search

Setting date and time

In [11]:
time_from = "2017-05-25T00:00:00Z"
time_to = "2017-10-25T23:59:59Z"

Setting search area

In [12]:
# Option 1: Set bounds as bbox
bounds = [
  12.742243,
  42.05043,
  12.746302,
  42.053218
]

# Option 2: Set bounds as Polygon
bounds = {
  "type": "Polygon",
  "coordinates": [
   [
    [
     12.500395,
     41.931337
    ],
    [
     12.507856,
     41.931018
    ],
    [
     12.507513,
     41.927825
    ],
    [
     12.50048,
     41.928719
    ],
    [
     12.500395,
     41.931337
    ]
   ]
  ]
 }

### AIRBUS Pleiades & SPOT

In [14]:
from CommercialDataBase import Providers, AirbusConstellation, ThumbnailType

# Option 1: AIRBUS Pleiades & SPOT
# Optional parameters: 
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#    - maxSnowCoverage (Vaules: 0-90 | 90 as default)
#    - maxIncidenceAngle (Values: 0-90 | 90 as default)
#    - processingLevel (Values: "Sensor","Album" | "Sensor","Album" as default)
#
#  example: shcd.search_airbus(AirbusConstellation, Bounds, Time_From, Time_To, *Optional parameters*)

(data, query) = shcd.search_airbus(AirbusConstellation.SPOT, bounds, time_from, time_to, maxCloudCoverage=90, maxSnowCoverage=50)

# Getting ids
item_ids = [feature["properties"]["id"] for feature in data.features]


DownloadFailedException: Failed to download from:
https://services.sentinel-hub.com/api/v1/dataimport/search
with HTTPError:
400 Client Error: Bad Request for url: https://services.sentinel-hub.com/api/v1/dataimport/search
Server response: "Cannot deserialize instance of `[D` out of START_OBJECT token
 at [Source: (org.glassfish.jersey.message.internal.ReaderInterceptorExecutor$UnCloseableInputStream); line: 1, column: 43] (through reference chain: com.sinergise.sentinel.dataimport.airbus.AirbusQuery["bounds"]->com.sinergise.sentinel.api.process.request.Bounds["bbox"])"

### Planet SCOPE

In [ ]:
from CommercialDataBase import ScopeType, ScopeBundle

# Option 2: Planet SCOPE
# Optional parameters: 
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#
#  example: shcd.search_airbus(ScopeType, ScopeBundle, Bounds, Time_From, Time_To, *Optional parameters*)

(data, query) = shcd.search_planet(ScopeType.PSScene, ScopeBundle.ANALYTIC_8B_UDM2, bounds, time_from, time_to, maxCloudCoverage=90)

# Getting ids
item_ids = [feature["id"] for feature in data.features]


### Planet SkySat

In [ ]:
from CommercialDataBase import SkySatType, SkySatBundle

# Option 3: Planet SkySat
#
#  example: shcd.search_airbus(ScopeType, ScopeBundle, Bounds, Time_From, Time_To, planetApiKey=<your_planey_api_key>)

(data, query) = shcd.search_planet(SkySatType.SkySatCollect, SkySatBundle.PANCHROMATIC, bounds, time_from, time_to, planetApiKey=<your_planey_api_key>)

# Getting ids
item_ids = [feature["id"] for feature in data.features]



### WorldView

In [ ]:
from CommercialDataBase import WorldViewKernel, WorldViewSensor

# Option 4: WorldView MAXAR
# Optional parameters:
#    - maxCloudCoverage (Values: 0-100 | 100 as default)
#    - minOffNadir (Values: 0-45 | 0 as default)
#    - maxOffNadir (Values: 0-45 | 45 as default)
#    - minSunElevation (Values: 0-90 | o as default)
#    - maxSunElevation (Values: 0-90 | 90 as default)
#    - sensor (Values: WorldViewSensor | Any as default)
#
#  example: shcd.search_airbus(WorldViewKernel, ScopeBundle, Bounds, Time_From, Time_To, *Optional parameters*)
#

(data, query) = shcd.search_worldview(WorldViewKernel.MTF, bounds, time_from, time_to, sensor=WorldViewSensor.WV01.value)

# Getting ids
item_ids = [feature["id"] for feature in data.features]



## Thumbnail

In [ ]:

item_id = item_ids[0]
thumbnail = shcd.thumbnail(ThumbnailType.SPOT, item_id)

import io
from PIL import Image

image_bytes = io.BytesIO(thumbnail.content)
Image.open(image_bytes)

# Data Order

### Query order: Airbus, Planet SkySat and Scope

In [4]:
response = shcd.order("New query order", "<your-collection-id>", query)


### Normal order: Airbus, Planet SkySat and Scope

In [4]:
response = shcd.order("New normal order", "<your-collection-id>", query, item_ids=item_ids)


### Getting orders

In [ ]:
# Optional: Get single order by adding order_id as parameter
# example: shcd.get_orders("asdf12-12bfa...")

orders = shcd.get_orders()
order_ids = [o["id"] for o in orders["data"]]
order_id = order_ids[0]

### Delete order

In [ ]:
shcd.delete_order(order_id)

### Confirm order

In [ ]:
shcd.confirm_order(order_id)